In [1]:
import pandas as pd
import numpy as np 
import datetime
import math
import gc
import datetime
gc.collect()

from tqdm import tqdm

# Load Data

In [2]:
# load play by play & drive data
years = list(range(2005, int(datetime.datetime.now().year)))

df = pd.DataFrame()
for year in tqdm(years):
    path = './output/'+str(year)+'/'+str(year)+'_pbp.csv'
    sea_df = pd.read_csv(path)
    
    drive_path = './output/'+str(year)+'/'+str(year)+'_drives.csv'
    drive_df = pd.read_csv(drive_path)
    
    drive_df = drive_df.rename(columns={'id':'drive_id'})
    
    sea_df = pd.merge(left=sea_df, right=drive_df, how='left', on=['drive_id','drive_id'])
    df = pd.concat([df,sea_df])

num_plays = len(df)
print(df.shape)
print(str(num_plays) + " plays were loaded")

100%|██████████| 14/14 [00:32<00:00,  3.04s/it]

(2031893, 41)
2031893 plays were loaded


In [3]:
# offense_x seems to be correct while offense_y is not

df = df.drop(columns=['defense_y','defense_conference_y','offense_y','offense_conference_y'])

df = df.rename(columns={'defense_x':'defense','defense_conference_x':'defense_conference','offense_x':'offense','offense_conference_x':'offense_conference'})


In [4]:
# fix yard_line, it's w.r.t the home team
df = df.rename(columns={'yard_line':'wrong_yardline'})

df['yard_line'] = np.where(df['offense']==df['home'],df['wrong_yardline'],100-df['wrong_yardline'])
# print(df[['home','offense','yard_line','wrong_yardline']].head(50))
df = df.drop(columns=['wrong_yardline'])

## Special Teams

I'll do special teams and overtimes in future work. Right now, keeping it simple.

In [5]:
print(len(df))
nah = ['2pt Conversion','Kickoff','Kickoff Return (Offense)','Kickoff Return Touchdown',
      'Offensive 1pt Safety','Defensive 2pt Conversion','Extra Point Good','Extra Point Missed']

df = df.loc[~df.play_type.isin(nah)]
print(len(df))

2031893
1871043


In [6]:
# also drop overtime

df = df.loc[(df.period > 0) & (df.period <= 4)]
print(len(df))


1864121


## Clock

Clock data is unreliable because maybe 25% of the games have only have one time for each play, and that time is when the drive started. I played with trying to predict time per play based on play type, but the data was very messy. So I decided to get the total time of each drive, and then assume each play took the same amount of time. EPA shouldn't be significantly affected most of the time, i.e. a 70 yard pass will be considered a good play no matter what. The only time it might have an adverse effect is toward the end of a game, when seconds matter. I think that in college football, when the clock stops for a first down, and incompletions, that all pass plays probably do take a somewhat similar amount of time. Drives in this situation will consist mostly of the same play type, and plays of the same play type likely take similar amounts of time. I'll compare it to the clock data I do have to make sure.

In [7]:
# fix clock data first so drives can be figured out
time_cols = ['clock.minutes','clock.seconds','start_time.minutes','start_time.seconds',
            'end_time.minutes','end_time.seconds']
for tc in time_cols:
    df[tc] = df[tc].fillna(0)

# get time remaining in game
df['tr_game'] = (4-df['period']) * 900 + (df['clock.minutes'] * 60) + df['clock.seconds']
df['tr_half'] = np.where(df['period']>2,df['tr_game'], df['tr_game']-1800)

df = df.drop(columns=['clock.minutes','clock.seconds'])

In [8]:
# fill empties
df['elapsed.minutes'] = df['elapsed.minutes'].copy().fillna(0)
df['elapsed.seconds'] = df['elapsed.seconds'].copy().fillna(0)
df['drive_time'] = 60*df['elapsed.minutes'] + df['elapsed.seconds']

In [9]:
# a lot of those drive times are negative... and other problems. so here's an alt drive time
# alt clock

# these get the start and end time of every drive
maxs = df.groupby(['game_id','drive_id'])['tr_game'].max().reset_index()
mins = df.groupby(['game_id','drive_id'])['tr_game'].min().reset_index()
maxs = maxs.rename(columns={'tr_game':'drive_start'})
mins = mins.rename(columns={'tr_game':'drive_end'})

# sometimes the drive end time is the same as the drive start. in that case, I use the next drive start
maxs = maxs.sort_values(by=['game_id','drive_start'],ascending=False)
next_max = maxs.groupby(['game_id'])['drive_start'].shift(-1)
next_max = pd.Series(next_max, name='next_drive_start')
new_max = pd.concat([maxs, next_max], axis=1)
new_max['next_drive_start'] = new_max['next_drive_start'].fillna(0)

# sometimes (rarely, 2%ish of the time) both the next drive start and the drive end are the same as the drive start
# in that case, as a last resort, i use the next drive end time. 
# i'm fairly sure most of the time it's when a timeout or something divides the same drive into two.
# i can explore this more in future work
mins = mins.sort_values(by=['game_id','drive_end'],ascending=False)
next_min = mins.groupby(['game_id'])['drive_end'].shift(-1)
next_min = pd.Series(next_min, name='next_drive_end')
new_min = pd.concat([mins, next_min], axis=1)
new_min['next_drive_end'] = new_min['next_drive_end'].fillna(0)
new_min = new_min.drop(columns='game_id')
times = pd.merge(left=new_max,right=new_min,on=['drive_id','drive_id'],how='left')


# attempt 1 (works on ~95.5% of data)
times['drive_time_1'] = times['drive_start']-times['next_drive_start']
# plan B (95.8% of data)
times['drive_time'] = np.where(times['drive_time_1']>0,times['drive_time_1'],(times['drive_start']-times['drive_end']))
# last resort (didn't implement)
# times['drive_time'] = np.where(times['drive_time_2']>0,times['drive_time_2'],(times['drive_start']-times['next_drive_end']))

not_good = times.loc[times.drive_time<=0]
print(len(not_good))

good = times.loc[times.drive_time>0]
print(len(good))

print(good.drive_time.mean())

times = times[['drive_id','drive_time']]
times = times.rename(columns={'drive_time':'alt_drive_time'})


8969
275461
146.12538980109707


In [10]:
df = pd.merge(left=df,right=times,how='left',on=['drive_id','drive_id'])


In [11]:
# longest drive in CFB history is 882. so need to drop anything above 900
# also drop anything below or equal to 0

df['correct_time_1'] = np.where(df['drive_time'] > 0, df['drive_time'], df['alt_drive_time'])

df = df.loc[df['correct_time_1'] > 0]

df['correct_drive_time'] = np.where(df['drive_time'] < 900, df['drive_time'], df['alt_drive_time'])

df = df.loc[df['correct_drive_time'] < 900]

print(len(df))

print("correlation between primary and approximate drive time")
print(df[['drive_time','alt_drive_time']].corr())

df = df.drop(columns=['drive_time','alt_drive_time','correct_time_1'])
df = df.rename(columns={'correct_drive_time':'drive_time'})

1844449
correlation between primary and approximate drive time
                drive_time  alt_drive_time
drive_time        1.000000        0.855643
alt_drive_time    0.855643        1.000000


# some objectives

1) fix "uncategorized" play type  
2) aggregate, clean, validate all play types  
3) fix "uncategorized" drive results  
4) aggregate, clean, validate all drive results  
5) compare play types to drive results to make sure they match

In [12]:
# fixing uncategorized play types

base = r'^{}'
expr = '(?=.*{})'
words = ['End', 'of', 'Quarter']
end_period = base.format(''.join(expr.format(w) for w in words))

df.loc[(df.play_type=='Uncategorized')&(df.play_text.str.contains(end_period,regex=True)), 'play_type'] = 'End Period'

words = ['fumbled','run', 'for']
fumbles = base.format(''.join(expr.format(w) for w in words))

df.loc[(df.play_type=='Uncategorized')&(df.play_text.str.contains(fumbles,regex=True)), 'play_type'] = 'Fumble Recovery (Own)'

df.loc[(df.play_type=='Uncategorized')&(df.play_text.str.contains('Penalty')), 'play_type'] = 'Penalty'


In [13]:
# fix individual
# many of the ones left are fumbles, and then something
df.loc[(df.play_type=='Uncategorized')&(df.play_text.str.contains('recovered by UTEP Aaron Jones')), 'play_type'] = 'Penalty'

df.loc[(df.play_type=='Uncategorized')&(df.play_text.str.contains('intercepted')), 'play_type'] = 'Pass Interception'

df.loc[(df.play_type=='Uncategorized')&(df.play_text.str.contains('SAFETY')), 'play_type'] = 'Safety'

words = ['fumbled','pass', 'complete']
complete = base.format(''.join(expr.format(w) for w in words))
df.loc[(df.play_type=='Uncategorized')&(df.play_text.str.contains(complete,regex=True)), 'play_type'] = 'Pass Completion'

words = ['TD','punt', 'blocked']
td_pb = base.format(''.join(expr.format(w) for w in words))
df.loc[(df.play_type=='Uncategorized')&(df.play_text.str.contains(td_pb,regex=True)), 'play_type'] = 'Blocked Punt Touchdown'


words = ['run','for', 'TD']
run_td = base.format(''.join(expr.format(w) for w in words))
df.loc[(df.play_type=='Uncategorized')&(df.play_text.str.contains(run_td,regex=True)), 'play_type'] = 'Rushing Touchdown'

df.loc[(df.play_type=='Uncategorized')&(df.down==4), 'play_type'] = 'Punt'

words = ['return','for', 'TD']
fumb_td = base.format(''.join(expr.format(w) for w in words))
df.loc[(df.play_type=='Uncategorized')&(df.play_text.str.contains(fumb_td,regex=True)), 'play_type'] = 'Fumble Return Touchdown'

df.loc[(df.play_type=='Uncategorized')&df.play_text.str.contains('return for'), 'play_type'] = 'Fumble Recovery (Opponent)'

df.loc[(df.play_type=='Uncategorized')&df.play_text.str.contains('run for'), 'play_type'] = 'Rush'

print("how many uncategorized plays are left?")
print(len(df.loc[df['play_type']=='Uncategorized']))


how many uncategorized plays are left?
0


In [14]:
nah_part_2 = ['End Period','End of Half','End of Game']
df = df.loc[~df.play_type.isin(nah_part_2)]

In [24]:
gb = df.groupby(['play_type'])['down'].count()
gb

play_type
Blocked Field Goal                       435
Blocked Field Goal Touchdown              35
Blocked Punt                             255
Blocked Punt Touchdown                   111
Field Goal Good                        24021
Field Goal Missed                       8758
Fumble Recovery (Opponent)              3671
Fumble Recovery (Own)                   4322
Fumble Return Touchdown                 1061
Interception                               2
Interception Return Touchdown            822
Missed Field Goal Return                  20
Missed Field Goal Return Touchdown         3
Pass                                   41599
Pass Completion                       225748
Pass Incompletion                     246884
Pass Interception                      11407
Pass Interception Return                6680
Pass Reception                        144382
Passing Touchdown                      13821
Penalty                               109836
Punt                                  107899


1) fix uncategorized play type (check)  
2) clean/validate/aggregate play types


In [16]:
# start with the top: blocked FG. make sure none are touchdowns
bfg = df.loc[(df.play_type=='Blocked Field Goal')&(df.play_text.str.contains('TOUCHDOWN|touchdown|Touchdown|TD'))]

print("There are a few!")
print(len(bfg))
print("All are defensive scores")

df.loc[((df.play_type=='Blocked Field Goal')&(df.play_text.str.contains('TOUCHDOWN|touchdown|Touchdown|TD'))), 'play_type']='Blocked Field Goal Touchdown'

del bfg

There are a few!
11
All are defensive scores


In [17]:
# same thing for blocked punt
bp = df.loc[(df.play_type=='Blocked Punt')&(df.play_text.str.contains('TOUCHDOWN|touchdown|Touchdown|TD'))]

print(len(bp))
print("punts to fix")

# pretty safe to assume these are all defensive
df.loc[((df.play_type=='Blocked Punt')&(df.play_text.str.contains('TOUCHDOWN|touchdown|Touchdown|TD'))), 'play_type']='Blocked Punt Touchdown'

print('fixed')
del bp

92
punts to fix


In [26]:
# regular FG missed, verify they weren't blocked/touchdown (all good)
# mfg = df.loc[(df.play_type=='FG Missed')&(df.play_text.str.contains('Blocked|BLOCKED|blocked'))]
# mfg = df.loc[(df.play_type=='FG Missed')&(df.play_text.str.contains('TOUCHDOWN|touchdown|Touchdown|TD'))]


0
missed fg to fix


In [29]:
# check Fumble Recovery (Opponent)
fro = df.loc[(df.play_type=='Fumble Recovery (Opponent)')&(df.play_text.str.contains('TOUCHDOWN|touchdown|Touchdown|TD'))]

print(len(fro))
print("fumble recoveries that were TDs")

# safe these were defensive
df.loc[(df.play_type=='Fumble Recovery (Opponent)')&(df.play_text.str.contains('TOUCHDOWN|touchdown|Touchdown|TD')), 'play_type'] = 'Fumble Return Touchdown'
for pt in list(fro.play_text.values):
    print(pt)


68
TEAM run for a loss of 13 yards TEAM fumbled, recovered by Vandy Oren Burks for a TD, (Hayden Lekacz KICK)
Mitch Trubisky pass complete to Quinshad Davis for 6 yds for a TD Quinshad Davis fumbled, forced by Nick Sigmon, recovered by Liber Jacob Hagen  , return for 27 yds, (Avery Echols KICK)
David Ash run for a loss of 1 yard David Ash fumbled, recovered by NoTex James Jones for a TD, (Trevor Moore KICK)
Jarod Neal pass complete to Ladevin Fair for 5 yds for a TD Ladevin Fair fumbled, recovered by Kent Khalid Henderson  , return for 89 yds, (Austin MacGinnis KICK)
Keenan Reynolds run for a loss of 2 yards Keenan Reynolds fumbled, forced by Sharif Finch, recovered by Temp Sharif Finch for a TD, (Austin Jones KICK)
Cedric Skinner run for a loss of 1 yard Cedric Skinner fumbled, forced by Earl Foster, recovered by Houst Vincent Hall for a TD, (Kyle Bullard KICK)
TEAM run for no gain TEAM fumbled, recovered by FlaAM John Boston for a TD
Arkeel Newsome run for a loss of 8 yards Arkeel Ne

In [19]:
# change rushing tds categorized as 'rush' to rushing tds
rush = df.loc[(df.play_type=='Rush')&(df.play_text.str.contains('TOUCHDOWN|touchdown|Touchdown|TD'))]
# for penalties that don't stop a touchdown
words = ['0 yard','accepted']
pens = base.format(''.join(expr.format(w) for w in words))
rush_tds = rush.loc[(~rush.play_text.str.contains('Penalty|PENALTY|penalty')) | (rush.play_text.str.contains('declined|DECLINED') | (rush.play_text.str.contains(pens)))]
rush_tds = rush_tds.loc[~rush.play_text.str.contains('fumbled')]
rtd_ids = list(rush_tds.id.values)
df.loc[df.id.isin(rtd_ids),'play_type'] = 'Rushing Touchdown'

del rush_tds
# rtd_ids = list(rush_tds.id.values)

In [20]:
# some fumble 6 rushes that need to be categorized as such

rush = df.loc[(df.play_type=='Rush')&(df.play_text.str.contains('TOUCHDOWN|touchdown|Touchdown|TD'))]
words = ['fumbled','returned by']
ftds = base.format(''.join(expr.format(w) for w in words))
fumble_tds = rush.loc[(rush.play_text.str.contains(ftds))]

fbtd_ids = list(fumble_tds.id.values)
df.loc[df.id.isin(fbtd_ids),'play_type'] = 'Fumble Return Touchdown'

words = ['fumbled','loss of']
ftds = base.format(''.join(expr.format(w) for w in words))
fumble_tds = rush.loc[(rush.play_text.str.contains(ftds))]

fbtd_ids = list(fumble_tds.id.values)
df.loc[df.id.isin(fbtd_ids),'play_type'] = 'Fumble Return Touchdown'

# subset of fumble 6s always say 'to the {other team} 0' 
words = ['to the','0']
ftds = base.format(''.join(expr.format(w) for w in words))
fumble_tds = rush.loc[(rush.play_text.str.contains(ftds))]

fbtd_ids = list(fumble_tds.id.values)
df.loc[df.id.isin(fbtd_ids),'play_type'] = 'Fumble Return Touchdown'

In [21]:
df.loc[((df.play_type=='Rush')&(df.play_text.str.contains('penalty|PENALTY|Penalty'))), 'play_type'] = 'Penalty'



In [22]:
# hard to determine which fumbles go for offensive TD vs defensive TD from just play text
# lean on drive result

df.loc[(df.play_type=='Rush')&(df.drive_result=='RUSHING TD'), 'play_type'] = 'Rushing Touchdown'

df.loc[(df.play_type=='Rush')&(df.drive_result=='FUMBLE RETURN TD'), 'play_type'] = 'Fumble Return Touchdown'

# i verified these
df.loc[((df.play_type=='Rush')&(df.yard_line>90)&(df.play_text.str.contains('TOUCHDOWN|touchdown|Touchdown|TD'))), 'play_type'] = 'Rushing Touchdown'

# rest seem to be defensive. might be one or two offensive that leaked through

df.loc[((df.play_type=='Rush')&(df.play_text.str.contains('TOUCHDOWN|touchdown|Touchdown|TD'))), 'play_type'] = 'Fumble Return Touchdown'
                               


In [23]:
print(list(df))

['away', 'defense', 'defense_conference', 'defense_score', 'distance', 'down', 'drive_id', 'home', 'id', 'offense', 'offense_conference', 'offense_score', 'period', 'play_text', 'play_type', 'yards_gained', 'season', 'week', 'season_type', 'drive_result', 'elapsed.minutes', 'elapsed.seconds', 'end_period', 'end_time.minutes', 'end_time.seconds', 'end_yardline', 'game_id', 'plays', 'scoring', 'start_period', 'start_time.minutes', 'start_time.seconds', 'start_yardline', 'yards', 'yard_line', 'tr_game', 'tr_half', 'drive_time']
